In [2]:
# Imports
import pandas as pandas_lib
import numpy as numpy_lib
import matplotlib.pyplot as mat_lib
import seaborn as seaborn_lib
import pickle 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import sigmoid_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from ast import literal_eval
import requests


data = pandas_lib.read_csv(r'D:\Coding\Recommendation engine\Data Files/clean_data.csv')
data['overview'] = data['overview'].fillna('')
indexes = data.index
title = data['original_title']

# indices is storing index of each movie
indices = pandas_lib.Series(indexes,index=data['original_title'])
features = ['cast','crew' , 'keywords','genres']
feat = ['id' , 'original_title']
mov_and_id = data[feat]


# literal_eval(data['crew'][0])
# for understanding only

for i in features:
    data[i] = data[i].apply(literal_eval)
    
def get_director(x):
    for i in x:
        if i['job']=='Director':
            return i['name'].strip()
    else:
        return numpy_lib.nan
    
data['director'] = data['crew'].apply(get_director)

def get_actor(x):
    names = [i['name'].strip().lower() for i in x if type(x)==list]
    # return only 3 or less names         
    if len(names)>3:
        return names[0:3]
    else:
        return names
        
for i in features:
    data[i] = data[i].apply(get_actor)
    
data.dropna(subset=['director'], inplace=True)

def create_feature(row):
    return ' '.join(row['keywords']) + ' ' + ' '.join(row['cast']) + ' ' + row['director'] + ' ' + ' '.join(row['genres'])

data['required_feature'] = data.apply(create_feature,axis=1)
cnt = CountVectorizer(stop_words='english')
count_matrix = cnt.fit_transform(data['required_feature'])
similarities = cosine_similarity(count_matrix,count_matrix)

In [3]:
similarities[0]

array([1.    , 0.1875, 0.1875, ..., 0.    , 0.    , 0.    ])

In [4]:
# pickle.dump(similarities,open('content_engine.pkl','wb'))

In [5]:
data.shape

(4773, 21)

In [6]:
similarities

array([[1.        , 0.1875    , 0.1875    , ..., 0.        , 0.        ,
        0.        ],
       [0.1875    , 1.        , 0.125     , ..., 0.        , 0.        ,
        0.        ],
       [0.1875    , 0.125     , 1.        , ..., 0.        , 0.16666667,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.16666667, ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])